<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/00-bow%2Bbinario%2Bsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline JCC: BoW+SVM

En esta notebook se presetan los experimentos sobre la estrategia de representación y técnica de aprendizaje *baseline* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- Pesado binario/no binario,
- Máquina de vector soporte (SVM).


In [1]:
# Cargamos el archivo con las consultas que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('03-Correos_variables_estaticas.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv

--2021-03-10 20:06:25--  https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 303767 (297K) [text/plain]
Saving to: ‘03-Correos_variables_estaticas.csv’

03-Correos_variable 100%[===================>] 296.65K  --.-KB/s    in 0.04s   

2021-03-10 20:06:25 (7.56 MB/s) - ‘03-Correos_variables_estaticas.csv’ saved [303767/303767]



In [2]:
# Leemos el archivo en un dataframe
import pandas as pd
df = pd.read_csv('03-Correos_variables_estaticas.csv', delimiter="|")

# Se transforma proveedor_correo a numerico
from sklearn import preprocessing
le_correo = preprocessing.LabelEncoder()
df['proveedor_correo'] = le_correo.fit_transform(df['proveedor_correo'])

In [3]:
# Me guardo los atributos, excepto la clase en x
x = df.drop(['Clase'], axis=1)
x.columns

Index(['Consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
       'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
       'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
       'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
       'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
       'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
       'cantidad_oraciones', 'utiliza_codigo_asignatura'],
      dtype='object')

In [4]:
# Transformamos todas las Clases minoritarias (Puedo ir variando la cantidad de clases que derivo a la Clase "Otras Consultas")
cantidad_clases=4

clases = df.Clase.value_counts()
clases_minoritarias = clases.iloc[cantidad_clases:].keys().to_list()

df.Clase[df['Clase'].isin(clases_minoritarias)] = "Otras Consultas"

# Clases balanceadas
df.Clase.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Otras Consultas             330
Boleto Universitario        240
Ingreso a la Universidad    232
Requisitos de Ingreso       129
Pedido de Certificados       69
Name: Clase, dtype: int64

In [5]:
# Se numeriza la clase
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Me quedo con las clases numerizadas
y=le.fit_transform(df['Clase'])

# Por otro lado me guardo las etiquetas de las clases
target_names=le.classes_

In [6]:
# Separo datos de entrenamiento y testing
from sklearn.model_selection import train_test_split

# Separo en 80-20 entrenamiento/validación y testeo
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.2)

## SVM

Generamos el modelo:

In [7]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

# Defino la feature y la transformación a aplicar en el texto
text_features = 'Consulta'
text_transformer = CountVectorizer()

# Defino la transformación
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features)
        ], remainder='passthrough')

# Combino la transformación con el pipeline
model_pipe = Pipeline([('preprocessor', preprocessor),
                       ('svm', SVC())])

# Defino los parámetros para GridSearchCV
parameters=[
        {'preprocessor__text__binary': [True, False],
         'preprocessor__text__analyzer': ['char'],
         'preprocessor__text__ngram_range': ((3, 3), (4, 7), (3, 4)),
         'preprocessor__text__strip_accents': ['unicode'],     
         'preprocessor__text__max_features':[500, 1000, 1500, 3000],
         'svm__C': [0.1, 1, 10, 100, 1000],  
         'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
         'svm__class_weight': [None, 'balanced'],
         'svm__kernel': ['rbf', 'linear', 'poly', 'rbf', 'sigmoid']
    }
]

# Instancio y "entreno" el GridSearchCV
grid_search=GridSearchCV(model_pipe, param_grid=parameters, cv=None, n_jobs=-1, verbose=3)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('text',
                                                                         CountVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.int64'>,
                      

In [8]:
# Imprimo los mejores parámetros encontrados por GridSearchCV
print(grid_search.best_params_) 
  
# Imprimo el modelo después del ajuste de hiperparámetros
print(grid_search.best_estimator_) 

{'preprocessor__text__analyzer': 'char', 'preprocessor__text__binary': False, 'preprocessor__text__ngram_range': (4, 7), 'svm__C': 1000}
Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  CountVectorizer(analyzer='char',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                             

In [9]:
from sklearn.metrics import classification_report, confusion_matrix 

# Se realizan las predicciones sobre el conjunto de validación
grid_predictions = grid_search.predict(x_test) 

# Se imprime el reporte de clasificación
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        47
           1       0.97      0.76      0.85        50
           2       0.65      0.97      0.78        66
           3       1.00      0.57      0.73        21
           4       0.78      0.44      0.56        16

    accuracy                           0.81       200
   macro avg       0.88      0.72      0.77       200
weighted avg       0.86      0.81      0.81       200



# Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05